# Ranked retrieval

Good for majority of users who wish to see results sorted by *relevance*.

## Approach 1: Jaccard coefficient
`Jaccard(A,B) = |A and B|/|A or B|`

### Pros
1. A and B dont have to be the same size
2. Always assigns a number b/w 0 and 1.

### Cons
1. Does not consider term frequency (we want to distinguish which term has high frequency vs low frequency) - the more frequent the term, the higher the relevance *should be*.

## Approach 2: Using term frequency and inverted document frequency

`tf = frequency of term d appearing in document d`  
Note: a document with 10x more freuency of this term does not mean that its 10x more relevant. (not proportional)

### Log frequency weighting

```
If (tf > 0)
    w = 1+log(tf)
else
    w = 0
```

Total score for a document-query pair:

`score = sum(1+log(tf))` from all terms in query AND document

### Document frequency

Rationale: Rare terms are more informative than frequent terms. eg. stop words concept  
`idf = log(N/df)`
- df = document frequency of term t, number of documents that contain the term t
- N is the total number of documents

Rarest/ low frequency words will have a larger `idf` value.

### Tf-idf weighting

`W = (1+log(tf))*log(N/df)`

#### Properties:
1. Increases with number of occurences within a document
2. Increases with rarity of the term in collection

#### Total score

`Score (q,d) = sum (tf-idf)` from all terms in query AND document

#### Issues:
1. We have a |V| dimensional space vector -> very large
2. Very sparse vectors, most are zero
3. No positional indexing and does not consider the ordering of words
4. If we feed a query with wrong word, we will get nothing

In [2]:
import nltk
import math


docs = {1:"who is aaryam, aaryam is a genius.", 
        2: "wilson is a hypocrite", 
        3: "tze guang is a guy who needs to chill is fab",
        4: "TTGHCS will become a reality, all hail Tze Guang",
        5: "Hit or miss, #road to 100 million for Pewds"
       }

query = input("input you query: ")
scores = {}
doc_freq = {}
doc_ranking = []

# Find document frequency of each term
for key in docs:
    text = docs[key]
    
    for term in list(set(nltk.word_tokenize(text))):
        if (term in doc_freq):
            doc_freq[term] += 1
        else:
            doc_freq[term] = 1
            

for key in docs:
    text = docs[key]
    term_freq = {}
    for term in nltk.word_tokenize(text):
        if (term not in term_freq):
            term_freq[term] = 1
        else:
            term_freq[term] += 1

    intersect = list(set(nltk.word_tokenize(text)) & set(nltk.word_tokenize(query)))
    
    # tf
    score_tf = sum(1 + math.log(term_freq[term]) for term in intersect)
    
    # idf
    score_idf = sum(math.log(len(docs)/float(doc_freq[term])) for term in intersect)
        
    print ("score for tf:", score_tf)
    print ("score for idf:", score_idf)
    print ("combined score:", score_tf*score_idf)
    doc_ranking.append((score_tf*score_idf, key))
    print ("=======================================================================")

print ("\nFinal result:")
doc_ranking.sort(reverse=True)
print (doc_ranking)


input you query: why hello there
score for tf: 0
score for idf: 0
combined score: 0
score for tf: 0
score for idf: 0
combined score: 0
score for tf: 0
score for idf: 0
combined score: 0
score for tf: 0
score for idf: 0
combined score: 0
score for tf: 0
score for idf: 0
combined score: 0

Final result:
[(0, 5), (0, 4), (0, 3), (0, 2), (0, 1)]


## Approach 3: text2vec cosine similarity model


Convert both query and document words into vectors. Perform tf.idf cosine similarity on them.

### Why euclidean distance is a bad idea

Distances in euclidean space are too large.  
Eg. a text with similar words similar distribution but one has much more word count. Their distances is very far apart even though they are similar in relevance

### Cosine similarity
Rank documents based on the angle distance to query.

#### Pros

1. Vectors are length normalised - making them comparable across different vectors despite their original lengths


In [2]:
import re, math
from collections import Counter
import nltk

def get_cosine(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection]) 
    # Tip: Perform these operations as matrix operations using numpy

    sum1 = sum([vec1[x]**2 for x in vec1.keys()])
    sum2 = sum([vec2[x]**2 for x in vec2.keys()])
    denominator = math.sqrt(sum1) * math.sqrt(sum2) # This makes it length normalised

    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator

def text_to_vector_ltc(text):
    words = nltk.word_tokenize(text)
    term_freq = Counter(words)
    term_score = {}

    # tf
    for term in list(set(words)):
        term_score[term] = 1 + math.log(term_freq[term])
    
    # idf
    for term in list(set(words)):
        term_score[term] *= math.log(len(words)/float(term_freq[term])) 
        # use len(words) as N to simulate each word as its own document
    return term_score


def text_to_vector_lnc(text):
    words = nltk.word_tokenize(text)
    term_freq = Counter(words)
    term_score = {}

    # tf
    for term in list(set(words)):
        term_score[term] = 1 + math.log(term_freq[term])
    
    # No idf
    return term_score



query = input("enter your query: ")
text2 = 'this sentence is similar to a foo bar sentence .'

vector1 = text_to_vector_ltc(query)
vector2 = text_to_vector_lnc(text2)

cosine = get_cosine(vector1, vector2)

print ('Cosine score:', cosine) # the larger the number, the more relevant


enter your query: this sentence is a foo bar sentence
Cosine score: 0.8347948681581757


## Additional stuff

Weighting can differ in queries vs document. One very popular weighting scheme is the `lnc.ltc` scheme.

### Document side (lnc):
- l: logarithmic tf
- n: no idf
- c: cosine normalisation

### Query side (ltc):
- l: logarithmic tf
- t: logarithmic idf
- c: cosine normalisation